In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import pickle



# the directory with all data
DATADIR = r".\adult" 
RESPONSEDIR = r".\adult\CastControls_ALP.xlsx"


# IMGWIDTH,IMGHEIGHT = (52, 70)
# IMGWIDTH,IMGHEIGHT = (130, 174)
# IMGWIDTH,IMGHEIGHT = (260, 348)
IMGWIDTH,IMGHEIGHT = (520, 696)

In [ ]:
# explaination
# pd.read_excel(io=RESPONSEDIR, sheet_name=1)

In [6]:
response = pd.read_excel(io=RESPONSEDIR, sheet_name=0)
response = response.dropna(axis=0, subset=['Grade'])
response.Grade = response.Grade.astype(int)

import cv2
df = pd.DataFrame(columns=['GreenID','img_array'],dtype='int64')
for image in os.listdir(DATADIR):
    if 'jpg' in image:
        img_array = cv2.imread(os.path.join(DATADIR, image), cv2.IMREAD_GRAYSCALE)
        # noticed they all have shape (520, 696)
        # problly resize if out of memory
    #     img_array = cv2.resize(img_array, (IMGHEIGHT, IMGWIDTH))
        df = df.append({'GreenID':int(re.search('[0-9]+', image).group(0)), 'img_array':img_array},ignore_index=True)

train = df.merge(response, how='inner', on='GreenID')
# want to remove either missing image or missing grade record

# filter out 'ungradables', and also the the poor quality ones (add them back later, maybe)
train_filter = train[~((train.Grade >= 6.01) | (train.Grade <= 0.99) )].sample(frac=1).reset_index()[['img_array', 'Grade','GreenID']]
# train_filter.img_array = train_filter['img_array'].apply(lambda x: np.array(x.flatten()))
# train_filter.img_array = train_filter.img_array.apply(lambda x: x.reshape(IMGWIDTH,IMGHEIGHT,1))

In [7]:
len(train_filter)

563

In [ ]:
train_filter.head(5)

In [ ]:
train_filter['img_array']

In [ ]:
# see if can be reconstructed (we want to convert to 1d array and normalise within each array)
# train_filter['img_array'].apply(lambda x: x.flatten().reshape(IMGWIDTH, IMGHEIGHT))[0]
fig, ax = plt.subplots(dpi=160)
plt.imshow(train_filter['img_array'].apply(lambda x: x.flatten().reshape(IMGWIDTH, IMGHEIGHT))[0],cmap='gray')
plt.show()

In [ ]:
train_filter[['img_array', 'Grade','GreenID']].to_pickle(r'.\processed_data\raw_data.pickle')

In [ ]:
# do some croppping then
import cv2 
df.img_array = df.img_array.apply(lambda x: x.reshape(IMGWIDTH,IMGHEIGHT, 1))

croped = pd.Series(list(np.array([tf.image.random_crop(
    df.img_array, (len(df.img_array), RE_SIZE, RE_SIZE ,1)).numpy() for i in range(40)]).reshape(-1, 250, 250, 1)))

grades = pd.Series(np.array([df.Grade.values for i in range(40)]).reshape(-1))
crop_df = pd.concat([croped,grades],axis=1)
# crop_df.to_pickle(r".\processed_data\cropped_data.pickle")

In [ ]:
######################################

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import pickle
import tensorflow as tf
DATADIR = r".\adult"
RESPONSEDIR = r".\CastControls_ALP.xlsx"


# IMGWIDTH,IMGHEIGHT = (52, 70)
# IMGWIDTH,IMGHEIGHT = (130, 174)
# IMGWIDTH,IMGHEIGHT = (260, 348)
IMGWIDTH,IMGHEIGHT = (520, 696)


train_filter = pd.read_pickle(r'.\processed_data\raw_data.pickle')

In [ ]:
cropped_imgs = tf.image.random_crop(value=np.stack(train_filter.head(5).img_array.values), size = (5,250,250,1)).numpy().reshape(-1,250,250)

In [ ]:
fig, ax = plt.subplots(dpi=160)
plt.imshow(cropped_imgs[0] ,cmap='gray')
plt.show()

In [ ]:
[tf.image.random_crop(value=np.stack(train_filter.head(5).img_array.values), size = (5,250,250,1)).numpy() for i in range(3)]

In [ ]:
# train[train.duplicated('ControlID', keep=False)].sort_values('ControlID',kind='mergesort')
# how to treat duplicated controlID? (correlation between inputs) maby be give less weight?

#### Now decide weight

In [ ]:
# if differ, treat as 2 cases but half weight; if same, double the weight
# maybe give less weight to the poor qualities
# does left or right matter?
# train_filter[~train_filter.RepeatGrade.isna()]

In [ ]:
X = np.array(np.stack(train_filter.img_array))

In [ ]:
img = np.random.randint(np.random.randint(563))
fig, ax = plt.subplots(dpi=160)
plt.imshow(X[img],cmap='gray')
plt.show()
print('GreenID: {} ; Rating: {}; Comments: {}'.format(train_filter.GreenID[img], train_filter.Grade[img], train_filter.CastStatusComments[img]))

In [ ]:
plt.imshow(X[2].reshape(-1,IMGWIDTH,IMGHEIGHT)[0],cmap='gray')
plt.show()

In [ ]:
PROCESSEDDIR = r".\processed_data"

# do not standardise/ohe yet
X_train = X.reshape(-1,IMGWIDTH,IMGHEIGHT,1)
y_train = train_filter.Grade.values

# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(handle_unknown='ignore',sparse=False, dtype=np.int)
# y_train = ohe.fit_transform(y_train.reshape(-1,1))

It is found out that fitting the entire image into the model is very inefficient; simply downscaling would also be very bad since the sample size is already extremely small. Maybe try random cropping first?